In [ ]:
# install dependencies
!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!
!pip install torch
!pip install trl
!pip install torchvision
!pip install peft
!pip install xformers


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-9w29d5pq
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-9w29d5pq
  Resolved https://github.com/unslothai/unsloth.git to commit 229e2ecc67756f36316dfcbea42396f59eef44e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.9.7-py3-none-any.whl size=315254 sha256=99f9236f1ff50abcc2d7d7cd9c056832f6af3e299089e0aeb5cad9b8014f963f
  Stored in directory: /tmp/pip-ephem-wheel-cache-4te48zsw/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.9.7
    Uninstalling unsloth-2025.9.7:
      Successfully uninstalled unsloth-2025.9.7


In [ ]:
# import all libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

In [ ]:
# check hf tokens
from google.colab import userdata
hf_token=userdata.get('hf_token')
login(token=hf_token)

In [ ]:
# check gpu availability
import torch
print("cuda avail:",torch.cuda.is_available())
print("gpu device",torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu")

cuda avail: True
gpu device Tesla T4


In [ ]:
# setup pretrained model
model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
max_seq_length=2048 # Corrected parameter name
dtype=None
load_in_4bit=True

model, tokenizer =FastLanguageModel.from_pretrained(
model_name=model_name,
max_seq_length=max_seq_length,
dtype=dtype,
load_in_4bit=load_in_4bit,
token=hf_token)

==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
# Step6: Setup system prompt
prompt_style = """
Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.

### Query:
{}

### Answer:
<think>{}
"""

In [ ]:
# run inference on the model
# define a test question
question="""A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or
              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""
FastLanguageModel.for_inference(model)
# tokenize the input
inputs=tokenizer([prompt_style.format(question, "")], return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in inputs.items()}
# generate a response
output=model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=1200,
    use_cache=True
)

# decode the response tokens back to text
reponse=tokenizer.decode(output[0], skip_special_tokens=True)
print(reponse[0])

In [ ]:
# now fine-tuning
# load-dataset
medical_dataset=load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split = "train[:500]", trust_remote_code=True)

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [ ]:
medical_dataset[0]

{'Question': 'Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?',
 'Complex_CoT': "Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the l

In [ ]:
# add a token beg of sentence and end of sentence
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which tells the model when to stop generating text during training
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [ ]:
### Finetuning
# Updated training prompt style to add  tag
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""


In [ ]:
# prepare the data for fine tuning
def preprocess_input_data(examples):
  inputs=examples['Question']
  cots=examples['Complex_CoT']
  outputs=examples['Response']
  texts=[]
  for input,cot,output in zip(inputs,cots,outputs):
    text=train_prompt_style.format(input,cot,output) + EOS_TOKEN
    texts.append(text)


  return{
      "texts": texts,
  }

In [ ]:
finetune_dataset=medical_dataset.map(preprocess_input_data, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
finetune_dataset['texts'][0]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nGiven the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?\n\n### Response:\n<think>\nOkay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is sw

In [ ]:
# setup/apply lora fintuning to the model
model_lora=FastLanguageModel.get_peft_model(
    model=model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3047,
    use_rslora=False,
    loftq_config=None
)



Unsloth 2025.9.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
def formatting_func(example):
    if isinstance(example["texts"], list):
        return example["texts"]
    else:
        return [example["texts"]]

trainer = SFTTrainer(
    model = model_lora,
    tokenizer = tokenizer,
    train_dataset = finetune_dataset,
    formatting_func = formatting_func,   # 👈 now safe for batching
    max_seq_length = max_sequence_length,
    dataset_num_proc = 1,

    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir = "outputs",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from google.colab import userdata
wnb_token=userdata.get("wandb")
# login to wnb
wandb.login(key=wnb_token)#import wnb
run=wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B',
    job_type='training',
    anonymous='allow'
)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2k23-cs2312094 (2k23-cs2312094-pranveer-singh-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
trainer_stats=trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explic

Unsloth: Will smartly offload gradients to save VRAM!


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.944200
20,1.427700
30,1.408100
40,1.365800
50,1.392200
60,1.371300


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


In [ ]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▁▁▂▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▁▁▁
total_flos,1.668884138360832e+16
train/epoch,0.96
train/global_step,60
train/grad_norm,0.25559
train/learning_rate,0.0
train/loss,1.3713


In [ ]:
# testing after fine tuning
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing
              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""
FastLanguageModel.for_inference(model_lora)
# tokenize the input
inputs=tokenizer([prompt_style.format(question, "")], return_tensors="pt")
inputs = {k: v.to("cuda") for k, v in inputs.items()}
# generate a response
output=model_lora.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=1200,
    use_cache=True
)

# decode the response tokens back to text
reponse=tokenizer.decode(output[0], skip_special_tokens=True)
print(reponse)


Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.

### Query:
A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing
              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?

### Answer:
<think>
Okay, let's think about this. We have a 61-year-old woman with a long history of involuntary urine loss whe

In [ ]:
print(reponse.split("### Answer:")[1])


<think>
Okay, let's think about this. We have a 61-year-old woman with a long history of involuntary urine loss whenever she coughs or sneezes, but she doesn't leak at night. That makes me think about something like stress urinary incontinence. So, let's consider what stress incontinence usually looks like. It's often due to weakened pelvic muscles and ligaments, and it can happen when there's an increase in abdominal pressure, like when you cough or sneeze.

Now, she's undergoing a gynecological exam and a Q-tip test. I recall that the Q-tip test is used to check for urethral obstruction, which is a different kind of incontinence. It's more about whether there's a physical blockage in the urethra. But in this case, her problem seems to be more about the muscles not working well, which points towards stress incontinence rather than a blockage.

So, if we're considering her symptoms, we should think about how stress incontinence might affect her bladder. Stress incontinence usually mea

In [ ]:
question = """A 59-year-old man presents with a fever, chills, night sweats, and generalized fatigue,
              and is found to have a 12 mm vegetation on the aortic valve. Blood cultures indicate gram-positive, catalase-negative,
              gamma-hemolytic cocci in chains that do not grow in a 6.5% NaCl medium.
              What is the most likely predisposing factor for this patient's condition?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
reponse = tokenizer.batch_decode(outputs)

print(reponse[0].split("### Answer:")[1])


<think>
Alright, let's think about this. We've got a 59-year-old guy with a fever, chills, and fatigue. That's pretty typical for something like an infection. Then, there's that vegetation on the aortic valve. That sounds like it could be an abscess or something similar. Now, what about the blood culture results?

Okay, they found gram-positive, catalase-negative, gamma-hemolytic cocci in chains. Hmm, that's interesting. These are a type of bacteria that can be tricky because they can grow in a variety of environments. But wait, they don't grow in a 6.5% NaCl medium. That's a clue! It suggests they're not the typical Streptococcus species, which usually grow in high salt conditions.

So, what's special about these bacteria? Well, they're known for being highly resistant to certain antibiotics. That's a red flag for antibiotic resistance. Now, thinking about their environment, if they don't grow in a 6.5% NaCl medium, they might be more likely to be found in a low-salt environment. Tha

In [ ]:
# In your Google Colab Notebook, after fine-tuning...
# The following code is for saving the model to a directory
# which can then be downloaded.
model_lora.save_pretrained("deepseek_r1_medical_finetune")
tokenizer.save_pretrained("deepseek_r1_medical_finetune")

# You can download the entire folder as a zip file.
# To do so, you'll need to use the following commands:
!zip -r /content/deepseek_r1_medical_finetune.zip /content/deepseek_r1_medical_finetune
from google.colab import files
files.download("/content/deepseek_r1_medical_finetune.zip")

  adding: content/deepseek_r1_medical_finetune/ (stored 0%)
  adding: content/deepseek_r1_medical_finetune/README.md (deflated 65%)
  adding: content/deepseek_r1_medical_finetune/adapter_model.safetensors (deflated 8%)
  adding: content/deepseek_r1_medical_finetune/chat_template.jinja (deflated 75%)
  adding: content/deepseek_r1_medical_finetune/tokenizer_config.json (deflated 96%)
  adding: content/deepseek_r1_medical_finetune/tokenizer.json (deflated 85%)
  adding: content/deepseek_r1_medical_finetune/adapter_config.json (deflated 57%)
  adding: content/deepseek_r1_medical_finetune/special_tokens_map.json (deflated 69%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>